In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision import datasets
from torchvision import transforms

import pytorch_lightning as pl
torch.set_float32_matmul_precision('high')

In [24]:
from dotenv import load_dotenv
import os

DATASET_PATH = os.environ.get('DATASET_PATH',"./data")
DATASET_PATH += "/celeba/img_align_celeba/"
BATCH_SIZE = 728 if torch.cuda.is_available() else 64
NUM_WORKERS = int(os.cpu_count() / 2)

In [25]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [26]:
DATASET_PATH

'/home/pranav-pc/projects/DATA//celeba/img_align_celeba/'

In [27]:
datasets.CelebA(root=DATASET_PATH)

RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [14]:
class Celeba(pl.LightningDataModule):
    def __init__(self, root, imgsz, batchsz, no_workers):
        super(Celeba, self).__init__()
        self.root = root
        self.imgsz = imgsz
        self.batchsz = batchsz
        self.num_workers = no_workers

        self.transform = transforms.Compose(
            [transforms.Resize(self.imgsz),
             transforms.CenterCrop(self.imgsz),
             # transforms.Grayscale(num_output_channels=1),
             #transforms.AutoAugment(), #imagenet
               
             transforms.ToTensor(),
            transforms.Normalize(mean,std)
            ]
        )

    def setup(self, stage):
        if stage == "fit" or stage is None:
            self.train_dataset = datasets.ImageFolder(
                root=self.root,transform=self.transform
            )
            # self.val_dataset = datasets.ImageFolder(
            #     root=os.path.join(self.root, "validation"),transform=self.transform
            # )

        # if stage == "test" or stage is None:
        #     self.test_dataset = datasets.ImageFolder(
        #         root=os.path.join(self.root, "test"),transform=self.transform
        #     )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batchsz,
            shuffle=True,
            num_workers=self.num_workers,
        )

    # def val_dataloader(self):
    #     return DataLoader(
    #         self.val_dataset,
    #         batch_size=self.batchsz,
    #         shuffle=False,
    #         num_workers=self.num_workers,
    #     )

    # def test_dataloader(self):
    #     return DataLoader(
    #         self.test_dataset,
    #         batch_size=self.batchsz,
    #         shuffle=False,
    #         num_workers=self.num_workers,
    #     )

In [15]:
IMAGE_SIZE = 64
ds = Celeba(DATASET_PATH,IMAGE_SIZE,BATCH_SIZE,NUM_WORKERS)

In [16]:
import matplotlib.pyplot as plt

# Sanity Check
ds.prepare_data()
ds.setup('fit')
dataloader = ds.train_dataloader()


# Create a dictionary to store one image per class
class_images = {}

# Iterate over the dataset and store one image per class
for image, label in ds.train_dataset:

    class_images[label] = image
    if len(class_images) == 10:  # Stop after storing one image per class
        break

# Display the images from each class
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, (label, image) in enumerate(class_images.items()):
    ax = axes[i // 5, i % 5]
    ax.imshow(image.permute(1,2,0))
    ax.set_title(f"Class: {label}")
    ax.axis('off')

plt.tight_layout()
plt.show()

FileNotFoundError: Couldn't find any class folder in /home/pranav-pc/projects/DATA//celeba/img_align_celeba.

In [7]:

class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()

        def _block(in_feature, out_feature, normalize=True):
            layers = [nn.Linear(in_feature, out_feature)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feature,0.8))
            layers.append(nn.LeakyReLU(0.01))
            return layers

        self.g_model = nn.Sequential(
            *_block(latent_dim, 128, normalize=False),
            *_block(128, 256),
            *_block(256, 512),
            *_block(512, 1024),
            nn.Linear(1024, img_shape),
            nn.Tanh()
        )

    def forward(self, x):
        return self.g_model(x)


In [8]:
class Discriminator(nn.Module):
    def __init__(self,in_feature):
        super(Discriminator, self).__init__()

        self.d_model = nn.Sequential(
                                 nn.Linear(in_feature,512),
                                 nn.LeakyReLU(negative_slope=0.2),
                                 nn.Linear(512,256),
                                 nn.LeakyReLU(0.2),
                                 nn.Linear(256,1),
                                 nn.Sigmoid())

    def forward(self,x):
        return self.d_model(x)

In [9]:
class GAN(pl.LightningModule):
    def __init__(self,latent_dim,img_shape,lr,b1,b2):
        super().__init__()

        self.save_hyperparameters()

        self.generator = Generator(latent_dim,img_shape)
        self.discriminator = Discriminator(img_shape)

        self.automatic_optimization = False

    def forward(self,z):
        return self.generator(z)

    def adversarial_loss(self,y_hat,y):
        return nn.functional.binary_cross_entropy(y_hat,y)

    def training_step(self,batch,batch_idx):
        imgs, _ = batch 

        # sample noise 
        z = torch.randn(imgs.shape[0],self.hparams.latent_dim).type_as(imgs)


        # ground truth 
        label_one = torch.ones(imgs.shape[0],1).type_as(imgs)
        label_zero = torch.zeros(imgs.shape[0],1).type_as(imgs)

        # Get optimizers
        g_opt, d_opt = self.optimizers()

        # Train Generator
        self.toggle_optimizer(g_opt)

        self.generated_img = self(z)

        #log 
        if not batch_idx % 100:
            sample_imgs = self.generated_img[:6]
            grid = torchvision.utils.make_grid(sample_imgs.view(-1,3,IMAGE_SIZE,IMAGE_SIZE))
            self.logger.experiment.add_image('generated_images',grid,self.current_epoch)
        
        g_loss = self.adversarial_loss(self.discriminator(self.generated_img),label_one)
        self.log('g_loss',g_loss,prog_bar=True)

        self.manual_backward(g_loss)
        g_opt.step()
        g_opt.zero_grad()
        self.untoggle_optimizer(g_opt)


        # Train discriminator
        self.toggle_optimizer(d_opt)
        # how well it can be label as real
        real_loss = self.adversarial_loss(self.discriminator(imgs.view(imgs.shape[0],-1)),label_one)
        # how well it can be label as fake
        fake_loss = self.adversarial_loss(self.discriminator(self.generated_img.detach()),label_zero)

        d_loss = (real_loss + fake_loss)/2
        self.log('d_loss',d_loss,prog_bar=True)

        self.manual_backward(d_loss)
        d_opt.step()
        d_opt.zero_grad()
        self.untoggle_optimizer(d_loss)


    def validation_step(self,batch):
        pass


    def configure_optimizers(self):
        g_opt = torch.optim.AdamW(self.generator.parameters(),lr=self.hparams.lr, betas=[self.hparams.b1,self.hparams.b2])
        d_opt = torch.optim.AdamW(self.discriminator.parameters(),lr=self.hparams.lr, betas=[self.hparams.b1,self.hparams.b2])            
            
        return [g_opt,d_opt], []

    def on_validation_batch_end(self):
        validation_noise = torch.randn(8,self.hparams.latent_dim).type_as(self.generator.model[0].weight)
                    
        #log 
        sample_imgs = self(validation_noise)
        grid = torchvision.utils.make_grid(sample_imgs.view(-1,3,IMAGE_SIZE,IMAGE_SIZE))
        self.logger.experiment.add_image('Validation_z',grid,self.current_epoch)
        

In [10]:
latent_dim, img_shape, lr, b1, b2 = 100, 3*IMAGE_SIZE*IMAGE_SIZE, 2e-4, 0.5, 0.999
model = GAN(latent_dim, img_shape, lr, b1,b2)

In [11]:
# model = torch.compile(model)

In [ ]:
trainer = pl.Trainer(min_epochs=1,
                        max_epochs=2000 ,
                        # precision='16-mixed',
                        enable_model_summary=False,
                        # callbacks=[pl.callbacks.EarlyStopping('d_loss',min_delta=0.0,patience=5)],
                        enable_checkpointing  = True,
                        log_every_n_steps=10
                    )
trainer.fit(model,ds)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/pranav-pc/.env/lib/python3.12/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: |                                               | 0/? [00:00<?, ?it/s]